Please God let this be the end of this NLP Course! A M E N


<div dir="rtl">
<h1>
طبقه‌بندی کلمات با استفاده از مدل HMM
</h1>

در این نوتبوک، با استفاده از دیتاست
<a href="https://huggingface.co/datasets/AmazonScience/massive/viewer/fa-IR"> آمازون </a>
به طبقه‌بندی کلمات می‌پردازیم. بدین شکل که تک تک کلمات مهم قصدشان شناسایی می‌شود یا none طلقی می شوند.
</div>

In [1]:
# ! pip uninstall numpy
! pip install datasets hazm numpy

Found existing installation: numpy 1.24.3
Uninstalling numpy-1.24.3:
  Would remove:
    /opt/homebrew/Caskroom/miniforge/base/envs/datastuff/bin/f2py
    /opt/homebrew/Caskroom/miniforge/base/envs/datastuff/bin/f2py3
    /opt/homebrew/Caskroom/miniforge/base/envs/datastuff/bin/f2py3.9
    /opt/homebrew/Caskroom/miniforge/base/envs/datastuff/lib/python3.9/site-packages/numpy-1.24.3.dist-info/*
    /opt/homebrew/Caskroom/miniforge/base/envs/datastuff/lib/python3.9/site-packages/numpy/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user
^C
ERROR: Operation cancelled by user


In [2]:
import numpy as np
import pandas as pd
import time
from pprint import pprint
from datasets import load_dataset
from hazm import WordTokenizer
from tqdm import tqdm

In [3]:
dataset = load_dataset("AmazonScience/massive", 'fa-IR')

In [4]:
pprint(dataset['train'][0])

{'annot_utt': 'مرا [date : جمعه] ساعت [time : نه صبح] بیدار کن',
 'id': '1',
 'intent': 48,
 'judgments': {'grammar_score': [4, 4, 4],
               'intent_score': [1, 1, 1],
               'language_identification': ['target', 'target', 'target'],
               'slots_score': [1, 1, 1],
               'spelling_score': [2, 2, 2],
               'worker_id': ['3', '8', '14']},
 'locale': 'fa-IR',
 'partition': 'train',
 'scenario': 16,
 'slot_method': {'method': ['translation', 'translation'],
                 'slot': ['time', 'date']},
 'utt': 'مرا جمعه ساعت نه صبح بیدار کن',
 'worker_id': '3'}


<br>
<h2 dir="rtl">
برای این که از داده‌های دیتاست استفاده کنیم، نیاز است تا هر داده را به صورت
(word, label)
درآوریم
</h2>

In [5]:
import re

def extract_bracket_parts(text):
    pattern = r'\[([^:]+)\s*:\s*([^]]+)\]'
    matches = re.findall(pattern, text)

    extracted_parts = []
    for match in matches:
        before_colon = match[0].strip()
        after_colon = match[1].strip()
        extracted_parts.append((before_colon, after_colon))

    return extracted_parts

این تابع، نسخه‌ی برچسب خورده و بدون برچسب یک جمله را می‌گیرد (مطابق حالتی که در دیتاست داریم) و سپس با فرمتی که در داک تمرین خواسته شده، به صورت لیستی از تاپل‌های (کلمه، برچسب) باز می‌گرداند

In [6]:
tokenizer = WordTokenizer()

def get_word_tag_tuples(utt: str, annot_utt: str):
  tokens = tokenizer.tokenize(utt)
  dic = {token: (None, index) for index, token in enumerate(tokens)}
  tagged_parts = extract_bracket_parts(annot_utt)
  for part in tagged_parts:
    tag = part[0]
    words = tokenizer.tokenize(part[1])
    for w in words:
      try:
        dic[w] = (tag, dic[w][1])
      except KeyError:
        # Sometimes hazm tokenizer doesn't do a very good job!
        return []
  sorted_parts = sorted(dic.items(), key=lambda x: x[1][1])
  return [(key, value[0]) for key, value in sorted_parts]



In [7]:
d = dataset['train'][0]
get_word_tag_tuples(d['utt'], d['annot_utt'])

[('مرا', None),
 ('جمعه', 'date'),
 ('ساعت', None),
 ('نه', 'time'),
 ('صبح', 'time'),
 ('بیدار', None),
 ('کن', None)]

In [8]:
train_tagged_words = [get_word_tag_tuples(d['utt'], d['annot_utt']) for d in dataset['train']]
train_tagged_words = [l for l in train_tagged_words if l]

In [9]:
print(train_tagged_words[0])
print(len(dataset['train']))
print(len(train_tagged_words))

[('مرا', None), ('جمعه', 'date'), ('ساعت', None), ('نه', 'time'), ('صبح', 'time'), ('بیدار', None), ('کن', None)]
11514
11506


In [10]:
train_tagged_words_unstructured = [x for sublist in train_tagged_words for x in sublist]

In [11]:
all_tags = {tag for all_tags in train_tagged_words for _, tag in all_tags}
print(all_tags)
vocab = {word for all_words in train_tagged_words for word, _ in all_words }
len(vocab)

{'media_type', 'house_place', 'song_name', 'joke_type', 'email_folder', 'podcast_name', 'meal_type', 'business_name', 'app_name', 'definition_word', 'artist_name', 'time_zone', 'currency_name', 'relation', 'transport_name', 'podcast_descriptor', 'personal_info', 'ingredient', 'email_address', 'transport_descriptor', 'game_type', 'movie_name', 'radio_name', 'sport_type', 'movie_type', 'place_name', 'coffee_type', 'date', 'transport_type', 'color_type', 'player_setting', 'timeofday', 'alarm_type', 'cooking_type', 'general_frequency', 'audiobook_name', 'order_type', 'music_album', 'music_genre', 'playlist_name', 'music_descriptor', 'news_topic', 'list_name', 'game_name', 'time', 'change_amount', 'weather_descriptor', None, 'food_type', 'event_name', 'business_type', 'device_type', 'transport_agency', 'drink_type', 'person', 'audiobook_author'}


5689

<h2 dir="rtl">
پایان آماده‌سازی دیتاست
</h2>
<br>
<hr>
<br>
<h2 dir="rtl">
الگوریتم Viterbi
</h2>


محاسبه‌ی احتمال emission

In [12]:
def word_given_tag(word, tag, train_bag):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)#total number of times the passed tag occurred in train_bag
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
#now calculate the total number of times the passed word occurred as the passed tag.
    count_w_given_tag = len(w_given_tag_list)


    return count_w_given_tag, count_tag

محاسبه‌ی احتمال transition


In [13]:
def t2_given_t1(t2, t1, train_bag):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return count_t2_t1, count_t1

محاسبه‌ی ماتریس transition

In [14]:
tags_matrix = np.zeros((len(all_tags), len(all_tags)), dtype='float32')
for i, t1 in enumerate(list(all_tags)):
    for j, t2 in enumerate(list(all_tags)):
        res = t2_given_t1(t2, t1, train_tagged_words_unstructured)
        tags_matrix[i, j] = res[0]/res[1]

print(tags_matrix)

[[0.42176038 0.         0.00122249 ... 0.         0.00244499 0.00122249]
 [0.         0.22352941 0.         ... 0.         0.         0.        ]
 [0.         0.         0.19135803 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.25       0.         0.        ]
 [0.         0.         0.         ... 0.         0.26546392 0.        ]
 [0.         0.         0.         ... 0.         0.         0.25      ]]


ماتریس احتمالات تگ‌ها

In [15]:
tags_df = pd.DataFrame(tags_matrix, columns = list(all_tags), index=list(all_tags))
display(tags_df)

,media_type,house_place,song_name,joke_type,email_folder,podcast_name,meal_type,business_name,app_name,definition_word,...,weather_descriptor,None,food_type,event_name,business_type,device_type,transport_agency,drink_type,person,audiobook_author
media_type,0.421760,0.000000,0.001222,0.000000,0.000000,0.000000,0.000000,0.009780,0.000000,0.000000,...,0.000000,0.535452,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002445,0.001222
house_place,0.000000,0.223529,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.764706,0.000000,0.000000,0.000000,0.003922,0.000000,0.000000,0.000000,0.000000
song_name,0.000000,0.000000,0.191358,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.734568,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
joke_type,0.000000,0.000000,0.000000,0.270833,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.729167,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
email_folder,0.000000,0.000000,0.000000,0.000000,0.418182,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.581818,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
podcast_name,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.642857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
meal_type,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030769,0.000000,0.000000,0.000000,...,0.000000,0.753846,0.030769,0.015385,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
business_name,0.069111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.454161,0.000000,0.000000,...,0.000000,0.454161,0.004231,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
app_name,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.163934,0.000000,...,0.000000,0.770492,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
definition_word,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.305810,...,0.000000,0.694190,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [16]:
def viterbi(words, train_bag):
    state = []
    T = list(set([pair[1] for pair in train_bag]))

    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = []
        for tag in T:
            if key == 0:
                transition_p = 1  # tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]

            # compute emission and state probabilities
            prb = word_given_tag(words[key], tag, train_tagged_words_unstructured)
            emission_p = prb[0]/prb[1]
            state_probability = emission_p * transition_p
            p.append(state_probability)

        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)]
        state.append(state_max)
    return list(zip(words, state))

<br>
<hr>

<h1 dir="rtl">
تست
<h1>
<h6 dir="rtl">
پیاده‌سازی به اتمام رسیده. با صدا زدن تابع viterbi می‌توانیم خروجی مدل را مشاهده کنیم.
</h2>

In [17]:
test = dataset['test'][0]
print(test['utt'])

print('what we want: ', test['annot_utt'])
print('our result: ', viterbi(tokenizer.tokenize(test['utt']), train_tagged_words_unstructured))

این هفته ساعت پنج صبح بیدارم کن
what we want:  [date : این هفته] ساعت [time : پنج صبح] بیدارم کن
our result:  [('این', 'date'), ('هفته', 'date'), ('ساعت', None), ('پنج', 'time'), ('صبح', 'time'), ('بیدارم', None), ('کن', None)]


In [18]:
def measure_accuracy(test_size: int = len(dataset['test'])):
  total, total_correct = 0, 0

  start = time.time()

  for i in range(test_size):
    test_data = dataset['test'][i]
    tagged_answer = get_word_tag_tuples(test_data['utt'], test_data['annot_utt'])
    our_result = viterbi(tokenizer.tokenize(test_data['utt']), train_tagged_words_unstructured)
    total_correct += len([i for i, j in zip(our_result, tagged_answer) if i == j])
    total += len(our_result)

  end = time.time()
  difference = end-start

  accuracy = total_correct/total
  print('Viterbi Algorithm Accuracy: ',accuracy*100)
  print(f' ** took {difference} second! **')


measure_accuracy(100)

Viterbi Algorithm Accuracy:  71.98067632850241
 ** took 94.63200902938843 second! **


In [20]:
def measure_accuracy_long(test_size: int = len(dataset['test'])):
  total, total_correct = 0, 0
  
  start = time.time()

  for i in tqdm(range(test_size)):
    test_data = dataset['test'][i]
    tagged_answer = get_word_tag_tuples(test_data['utt'], test_data['annot_utt'])
    our_result = viterbi(tokenizer.tokenize(test_data['utt']), train_tagged_words_unstructured)
    total_correct += len([i for i, j in zip(our_result, tagged_answer) if i == j])
    total += len(our_result)
  
  end = time.time()
  difference = end-start

  accuracy = total_correct/total
  print('\nViterbi Algorithm Accuracy: ',accuracy*100)
  print(f' ** took {difference} second! **')

In [21]:
measure_accuracy_long()

100%|██████████| 2974/2974 [54:06<00:00,  1.09s/it]  


Viterbi Algorithm Accuracy:  69.57140234948605
 ** took 3246.4367003440857 second! **
